In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import os
import scipy.io
import scipy.signal
import mne
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
import time
from functools import partial
import torch
from torch.utils.data import DataLoader, Dataset, random_split

In [ ]:
from signal_extraction import SignalExtraction
import pickle

root_path = '/content/drive/MyDrive/'
file_path_openneuro = 'EEG/'
file_path = root_path + file_path_openneuro

start = time.time()
ls = SignalExtraction.read_mne_iterator(file_path, None)
end = time.time()
print(f"Data extraction time: {end - start}")

# Save the data to a pickle file
with open('extracted_signals.pkl', 'wb') as f:
    pickle.dump(ls, f)

<<<<<<<Processing /content/drive/MyDrive/EEG/sub-007/eeg>>>>>>>
Reading /content/drive/MyDrive/EEG/sub-007/eeg/sub-007_task-Rest_run-02_eeg.fdt
<<<<<<<Processing /content/drive/MyDrive/EEG/sub-010/eeg>>>>>>>
<<<<<<<Processing /content/drive/MyDrive/EEG/sub-115/eeg>>>>>>>
Reading /content/drive/MyDrive/EEG/sub-115/eeg/sub-115_task-Rest_run-02_eeg.fdt
<<<<<<<Processing /content/drive/MyDrive/EEG/sub-060/eeg>>>>>>>
Reading /content/drive/MyDrive/EEG/sub-060/eeg/sub-060_task-Rest_run-02_eeg.fdt
<<<<<<<Processing /content/drive/MyDrive/EEG/sub-066/eeg>>>>>>>
Reading /content/drive/MyDrive/EEG/sub-066/eeg/sub-066_task-Rest_run-02_eeg.fdt
<<<<<<<Processing /content/drive/MyDrive/EEG/sub-111/eeg>>>>>>>
Reading /content/drive/MyDrive/EEG/sub-111/eeg/sub-111_task-Rest_run-02_eeg.fdt
<<<<<<<Processing /content/drive/MyDrive/EEG/sub-088/eeg>>>>>>>
Reading /content/drive/MyDrive/EEG/sub-088/eeg/sub-088_task-Rest_run-02_eeg.fdt
<<<<<<<Processing /content/drive/MyDrive/EEG/sub-016/eeg>>>>>>>
Reading 

/content/signal_extraction.py:108: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  mne_obj = mne.io.read_raw_eeglab(mne_file)


<<<<<<<Processing /content/drive/MyDrive/EEG/sub-001/eeg>>>>>>>
Reading /content/drive/MyDrive/EEG/sub-001/eeg/sub-001_task-Rest_run-02_eeg.fdt
Data extraction time: 172.31960248947144


In [ ]:
import numpy as np
import pickle
from signal_processing import SignalProcessing
import time

# Load extracted signals from the pickle file
with open('extracted_signals.pkl', 'rb') as f:
    ls = pickle.load(f)

sliceSize = 6
bandFreqRange = [40, 100]
downsampleFreq = 200
subSliceSize = 2
sampleFrequency = 500

start = time.time()
training_data, error_data = SignalProcessing.signal_processing_iterator(ls, sampleFrequency, bandFreqRange, downsampleFreq, sliceSize, subSliceSize)
end = time.time()
print(f"Signal processing time: {end - start}")

print(f"Number of training data samples: {len(training_data)}")

# Save processed data for further preparation
with open('processed_signals.pkl', 'wb') as f:
    pickle.dump(training_data, f)

with open('error_signals.pkl', 'wb') as f:
    pickle.dump(error_data, f)


Signal processing time: 11.167258977890015
Number of training data samples: 33


In [ ]:
import numpy as np
import os
from training_data import TrainingData

# Load processed signals
training_data = np.load('processed_signals.pkl', allow_pickle=True)

# Creating a folder to save processed data
output_folder = "processed_data"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Saving processed data for each subject
for subject_id, signal_processed, Adj_dist_matrix, bdi, bdi_label in training_data:
    subject_folder = os.path.join(output_folder, subject_id)
    if not os.path.exists(subject_folder):
        os.makedirs(subject_folder)

    # Save processed data in the subject's folder
    np.save(os.path.join(subject_folder, "signal_processed.npy"), signal_processed)
    np.save(os.path.join(subject_folder, "Adj_dist_matrix.npy"), Adj_dist_matrix)
    np.save(os.path.join(subject_folder, "bdi_label.npy"), bdi_label)

X = TrainingData.trainingData_iterator(training_data)

# Distribution of categorical Variable
ls = []
for i in X:
    signal_slice, Adj_dist_matrix, bdi_label = i
    ls.append(bdi_label)

print(len(ls), sum(ls), len(ls)-sum(ls))

dataSet = X
print(len(dataSet))

from torch.utils.data import DataLoader, random_split

def train_test_datasets(dataSet, batch_size, train_test_split):
    train_size = int(len(dataSet) * train_test_split)
    test_size = len(dataSet) - train_size
    train_dataset, test_dataset = random_split(dataSet, [train_size, test_size])

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, test_dataloader

batch_size = 40
train_test_split = 0.7

train_dataloader, test_dataloader = train_test_datasets(dataSet, batch_size, train_test_split)

print(len(train_dataloader), len(test_dataloader), len(train_dataloader.dataset) + len(test_dataloader.dataset), (len(train_dataloader)-1)*batch_size + len(test_dataloader))

signal, Adj_dist_matrix, label = dataSet[0]
print(signal.shape, Adj_dist_matrix.shape, label, len(dataSet))

3268 2357 911
3268
58 25 3268 2305
(3, 16, 400) (16, 16) 0 3268
